<hr>

# **Stock Market Sentimental Prediction with OpenAI**

<hr>


**Today in the stock market, staying informed about news and events is crucial for making strategic decisions. ecognizing the impact of sentiment on market trends is essential to adjust strategies accordingly. The process begins with accessing vast amounts of market news available through various sources.**

<hr>

# **Step 1: Importing of the needed libraries**

In [3]:
%pip install openai
%pip install langchain
%pip install eodhd
%pip install config
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import re
import pandas as pd
import config as cfg
import requests as rq
from eodhd import APIClient
from dotenv import load_dotenv
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

<hr>

**We can proceed to the next step which is activating the API key.**

<hr>

In [11]:
try:
    load_dotenv()
except FileNotFoundError:
    print("Warning: Could not find .env file. Skipping token loading.")

# Retrieving the token from the environment variable securely
try:
    token = os.environ.get("API_TOKEN")
except KeyError:
    print("Error: Missing expected environment variable `{}`.".format("YOUR_TOKEN_VARIABLE_NAME"))
    raise


In [13]:
# initializing the API
api = APIClient(token)

**Extracting the data out of the API Key**

In [14]:
resp = api.financial_news(s = 'AAPL.US', from_date='2024-01-01', to_date='2024-01-30', limit=100)

resp

[{'date': '2024-01-30T22:24:05+00:00',
  'title': 'Dow Jones Futures: Microsoft, Google, AMD Fall On Earnings; Elon Musk Loses $56 Billion Pay Plan',
  'content': "Microsoft, Google and AMD earnings are a big test for AI, cloud and techs broadly. A judge struck down Elon Musk's huge pay plan.\n\nContinue reading",
  'link': 'https://finance.yahoo.com/m/c78e3618-8015-3089-b0cc-fe9c45f6fc2a/dow-jones-futures%3A-microsoft%2C.html',
  'symbols': ['0QYP.IL',
   '0R2V.IL',
   '0RIH.IL',
   'A1MD34.SA',
   'AAPL.BA',
   'AAPL.MX',
   'AAPL.NEO',
   'AAPL.SN',
   'AAPL.US',
   'AAPL34.SA',
   'ABE0.F',
   'ABEA.F',
   'ABEA.XETRA',
   'ABEC.F',
   'ABEC.STU',
   'ABEC.XETRA',
   'AMD.BE',
   'AMD.DU',
   'AMD.F',
   'AMD.HM'],
  'tags': [],
  'sentiment': {'polarity': -0.026, 'neg': 0.127, 'neu': 0.787, 'pos': 0.086}},
 {'date': '2024-01-30T21:29:00+00:00',
  'title': 'Top Stock Reports for Apple, Visa &amp; Novo Nordisk',
  'content': "Tuesday, January 30, 2024\r\n\r\nThe Zacks Research Daily

**Here we have got a JSON response, need to convert the same into a dataframe**

In [15]:
df = pd.DataFrame(resp)
df.tail()

,date,title,content,link,symbols,tags,sentiment
95,2024-01-26T15:05:00+00:00,Apple Lags as Microsoft Sales Hit Highs: US Ea...,(Bloomberg) -- Five of the so-called Magnifice...,https://finance.yahoo.com/news/apple-lags-micr...,"[A1MD34.SA, AAPL.MX, AAPL.US, AAPL34.SA, ABEA....",[],"{'polarity': 0.997, 'neg': 0.06, 'neu': 0.814,..."
96,2024-01-26T15:02:46+00:00,WR Berkley Corp Leads as Scott Black's Top Por...,Delphi Management's President Scott Black (Tra...,https://finance.yahoo.com/news/wr-berkley-corp...,"[0HMZ.LSE, 1NBA.BE, 1NBA.F, 1NBA.MU, 1NBA.STU,...",[],"{'polarity': 0.995, 'neg': 0.013, 'neu': 0.87,..."
97,2024-01-26T15:00:00+00:00,Intuitive Surgical (ISRG) Seeks FDA Nod for Ne...,Intuitive Surgical ISRG announced that it has ...,https://finance.yahoo.com/news/intuitive-surgi...,"[0R29.LSE, AAPL.MX, AAPL.US, AAPL34.SA, APC.F,...",[],"{'polarity': 0.99, 'neg': 0.04, 'neu': 0.845, ..."
98,2024-01-26T14:46:00+00:00,Electronic Arts (EA) to Report Q3 Earnings: Wh...,Electronic Arts EA is set to report third-quar...,https://finance.yahoo.com/news/electronic-arts...,"[0IFX.LSE, AAPL.MX, AAPL.US, AAPL34.SA, ABEA.F...",[],"{'polarity': 0.998, 'neg': 0.013, 'neu': 0.871..."
99,2024-01-26T14:10:00+00:00,Microsoft Is the World’s Most Valuable Company...,The company is in the early stages of infusing...,https://finance.yahoo.com/m/606b182c-5a22-3978...,"[AAPL.MX, AAPL.US, AAPL34.SA, ABEA.F, ABEA.XET...",[],"{'polarity': 0, 'neg': 0, 'neu': 1, 'pos': 0}"


**Parameters that are used on to over here**

* `s`: String. REQUIRED if parameter 't' is not set. The ticker code to get news for.

* `t`: String. REQUIRED if parameter 's' is not set. The tag to get news on a given topic. 

* `api_token`: String. REQUIRED. Your api_token to access the API. You will get it after registration.

* `from and to`: the format is 'YYYY-MM-DD'. If you need data from Mar 1, 2021, to Mar 10, 2021, you should use from=2021–03–01 and to=2021–03–10.

* `limit`: Number. OPTIONAL. The number of results should be returned with the query. Default value: 50, minimum value: 1, maximum value: 1000.

* `offset`: Number. OPTIONAL. The offset of the data. Default value: 0, minimum value: 0. For example, to get 100 symbols starting from 200 you should use limit=100 and offset=200.

<hr>

**The output results of the dataframe are like this:**

* **`date`: The date and time of the article are in ISO 8601 format.**
* **`title`: The title of the article.**
* **`content`: The full body of the article.**
* **`link`: The link to the source.**
* **`symbols`: The array of ticker symbols is mentioned in the article.**



<hr>

# **Step 2 - Data Cleaning**

In [16]:
# function for clearing the textual data

def clean_text(text):
    cleaned_text = re.sub('r\s+', ' ', text)
    return cleaned_text

df['content'] = df['content'].apply(clean_text)

**Now we have applied it to the complete data hence we can move forward to our chatbot**

In [ ]:
llm = ChatOpenAI(model = 'gpt-2.5-turbo',
                 openai_key = '')